In [ ]:
#################################
# csv 파일 불러와서 DB table에 넣기
# 데이터: 지하철 승하차
https://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do#
#################################

# 0. 데이터베이스 생성 (MySQL worchbench에서)
# 1. 데이터베이스 테이블 생성
# 2. pandas에서 csv 파일 불러오기
# 3. insert_into_table 함수 수정해서 csv 파일의 내용을 table에 저장
# 4. read_table 함수를 이용해서 저장된 내용 확인 (worchbench로 먼저 확인)
#    read_table 함수에는 limit 값 입력 받아서(ex 10,10)

# ENV

In [ ]:
# !pip install pandas

In [2]:
import glob

import pandas as pd
import pymysql

# 데이터 확인 - 생략 가능

In [4]:
filename = 'CARD_SUBWAY_MONTH_202311.csv'
df = pd.read_csv(filename, index_col=False)
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20231101,8호선,단대오거리,13482,13072,20231104
1,20231101,중앙선,원덕,372,362,20231104
2,20231101,중앙선,용문,2194,2097,20231104
3,20231101,중앙선,지평,72,71,20231104
4,20231101,우이신설선,4.19민주묘지,4515,4447,20231104


In [6]:
df['사용일자'].dtype

dtype('int64')

In [7]:
max(map(len, df['노선명'].unique()))

8

In [8]:
max(map(len, df['역명'].unique()))

16

In [10]:
max(df['승차총승객수']), max(df['하차총승객수'])

(93897, 101401)

In [38]:
df.iloc[0].iloc[0]

20231101

In [12]:
len(df)

18313

# 함수 정의

In [4]:
def create_table(cur, conn):
    # database에 table을 새로 생성
    global table_name
    table_name = input('테이블명 ==> ')
    # lst = []
    # while True:
    #     data1 = input('열이름 ==> ')
    #     if data1 == "":
    #         break
    #     data2 = input('데이터타입 ==> ')

    #     lst.append(data1+' '+data2)
    # create_str = f"create table {data0} ({', '.join(lst)})"
    create_str = f"create table {table_name} (사용일자 int, 노선명 char(10), 역명 char(20), 승차총승객수 int, 하차총승객수 int, 등록일자 int)"
    cur.execute(create_str)
    conn.commit()
    # conn.close()


def insert_into_table(df, table_name, cur, conn):
    for i in range(len(df)):
        row = df.iloc[i]
        data1 = row['사용일자']
        data2 = row['노선명']   # char
        data3 = row['역명']     # char
        data4 = row['승차총승객수']
        data5 = row['하차총승객수']
        data6 = row['등록일자']
        insert_sql = (
            f"insert into {table_name} values({data1}, '{data2}', '{data3}', {data4}, {data5}, {data6})"
        )
        cur.execute(insert_sql)
        # print(insert_sql)
        
    conn.commit()
    # conn.close()


# 390 page : 데이터 조회
def read_table(cur, table_name, start_idx = 0, how_many = 10):
    cur.execute(f"select * from {table_name} limit {start_idx}, {how_many}")

    print("=======================================================")

    while True:
        row = cur.fetchone()
        if row == None:
            break
        print(row)  # tuple ('hong', '홍지윤', 'hong@naver.com', 1996)
        # data1, data2, data3, data4 = row
        # print("%5s %15s %20s %d" % (data1, data2, data3, data4))

# CSV files 2 MySQL table

In [3]:
# 여러 csv 파일 사용
csv_lst = glob.glob('*.csv')
csv_lst

['CARD_SUBWAY_MONTH_2015.csv',
 'CARD_SUBWAY_MONTH_2016.csv',
 'CARD_SUBWAY_MONTH_2017.csv',
 'CARD_SUBWAY_MONTH_2018.csv',
 'CARD_SUBWAY_MONTH_2019.csv',
 'CARD_SUBWAY_MONTH_2020.csv',
 'CARD_SUBWAY_MONTH_2021.csv',
 'CARD_SUBWAY_MONTH_2022.csv',
 'CARD_SUBWAY_MONTH_202301.csv',
 'CARD_SUBWAY_MONTH_202302.csv',
 'CARD_SUBWAY_MONTH_202303.csv',
 'CARD_SUBWAY_MONTH_202304.csv',
 'CARD_SUBWAY_MONTH_202305.csv',
 'CARD_SUBWAY_MONTH_202306.csv',
 'CARD_SUBWAY_MONTH_202307.csv',
 'CARD_SUBWAY_MONTH_202308.csv',
 'CARD_SUBWAY_MONTH_202309.csv',
 'CARD_SUBWAY_MONTH_202310.csv',
 'CARD_SUBWAY_MONTH_202311.csv']

In [5]:
# 1. DB 연결
conn = pymysql.connect(host="127.0.0.1", user="root", password="0000", db="subway", charset="utf8")

# 2. 커서 생성
cur = conn.cursor()

In [6]:
# 3. table 생성
create_table(cur, conn)

In [7]:
# table name 확인
# table_name = 'subway'
table_name

'subway'

In [8]:
# 4. tahle에 데이터 넣기
# insert_into_table(df, table_name, cur, conn)
# # 여러 csv 파일 다 한 테이블에 넣기
for c in csv_lst:
    df = pd.read_csv(c, index_col=False, encoding='utf-8')
    insert_into_table(df, table_name, cur, conn)

In [9]:
# 5. table 읽어오기
read_table(cur, table_name)

(20150101, '2호선', '낙성대', 14586, 14889, 20151217)
(20150101, '2호선', '사당', 19233, 20298, 20151217)
(20150101, '2호선', '방배', 5920, 6065, 20151217)
(20150101, '2호선', '서초', 4379, 4120, 20151217)
(20150101, '분당선', '선정릉', 1972, 1828, 20151217)
(20150101, '분당선', '가천대', 2512, 3332, 20151217)
(20150101, '분당선', '태평', 7637, 7899, 20151217)
(20150101, '분당선', '모란', 11155, 11269, 20151217)
(20150101, '분당선', '야탑', 13041, 13379, 20151217)
(20150101, '분당선', '서현', 12399, 12379, 20151217)


In [10]:
read_table(cur, table_name, start_idx=500000, how_many=5)

(20170621, '6호선', '응암', 20379, 19917, 20170624)
(20170621, '6호선', '역촌', 4389, 5189, 20170624)
(20170621, '6호선', '불광', 6065, 6083, 20170624)
(20170621, '6호선', '독바위', 3404, 3548, 20170624)
(20170621, '6호선', '연신내', 1, 0, 20170624)


In [11]:
# 6. 연결 끊기
conn.close()

# Troubleshooting

In [ ]:
# 에러: 2020년 파일 71421번째 줄부터 없던 ,"" 이 생김 => replace_in_file함수로 없애주면서 인코딩 바꿈
# 에러: 2021년 파일 200593번째 줄 보니까, 두줄이 붙어있음 => 셀프 수정(메모장에서 엔터침)

In [13]:
# 한 파일 내에서 , 개수가 다름
# "20200430","8호선","암사","10113.0","8961.0","20200503"
# "20200501","중앙선","운길산","1349","1358","20200504",""
# 모든 파일 읽어와서 ,"" 없애기


def replace_in_file(file_path, old_str, new_str):
    # 파일 읽어들이기
    fr = open(file_path, 'r', encoding='utf-8')
    lines = fr.readlines()
    fr.close()
    
    # old_str -> new_str 치환
    fw = open(file_path, 'w', encoding='utf-8')
    for line in lines:
        fw.write(line.replace(old_str, new_str))
    fw.close()

# csv 파일에서 ,"" 없애기
for c in csv_lst:
    replace_in_file(c, ',""', '')

In [12]:
# 길이 확인
total = 0
for c in csv_lst:
    df = pd.read_csv(c, index_col=False, encoding='utf-8')
    total += len(df)
total

1898875